In [31]:
import numpy as np
import math
import matplotlib.pyplot as plt
import imageio
from tqdm.notebook import tnrange, tqdm_notebook

## Set parameters

In [32]:
#u = 5e-7
u = 0
#chi = 0
chi = 8e-12

dx = 4e-7
dt = 0.01

s = u * dt / dx
r = chi * dt / (dx ** 2)


teta = 5
a = 0
b = 1e-4

K = int((b - a) / dx) + 1

start = np.zeros(K, dtype=np.float64)
for i in range(K // 2):
    start[i] = 1.

## Define functions

In [36]:
def solve_linear(a, b, c, d):
    nf = len(d)
    ac, bc, cc, dc = map(np.array, (a, b, c, d))
    for it in range(1, nf):
        mc = ac[it - 1] / bc[it - 1]
        bc[it] = bc[it] - mc * cc[it - 1]
        dc[it] = dc[it] - mc * dc[it - 1]
    xc = bc
    xc[-1] = dc[-1] / bc[-1]

    for il in range(nf - 2, -1, -1):
        xc[il] = (dc[il] - cc[il] * xc[il + 1]) / bc[il]

    return xc

def get_dt(dx, u, chi, coeff=0.9):
    return coeff * (dx ** 2) / (u * dx + 2 * chi)




def left_border(t):
    return 0


def right_border(t):
    return 0


def solve(solver):
    t_iterations = int(teta / dt) + 1

    Ts = []
    Ts.append(start)

    for cur_t_iter in range(1, t_iterations + 1):
        t = dt * cur_t_iter
        cur = solver(t, Ts)
        Ts.append(cur)

    return Ts


def explicit_opposite(t, Ts):
    cur = np.zeros(K, dtype=np.float64)

    for i in range(K):
        prev_t = Ts[-1][i - 1] if i > 0 else left_border(t)
        next_t = Ts[-1][i + 1] if i < K - 1 else right_border(t)
        cur_t = Ts[-1][i]

        cur[i] = cur_t + r * (prev_t + next_t - 2 * cur_t)

    return cur


def explicit_nonopposite(t, Ts):
    cur = np.zeros(K, dtype=np.float64)

    for i in range(K):
        prev_t = Ts[-1][i - 1] if i > 0 else left_border(t)
        next_t = Ts[-1][i + 1] if i < K - 1 else right_border(t)
        cur_t = Ts[-1][i]

        cur[i] = cur_t - s * (next_t - cur_t) + r * (prev_t + next_t - 2 * cur_t)

    return cur


def confusion(t, Ts):
    #assert math.isclose(chi, 0.)

    if len(Ts) == 1:
        return explicit_opposite(t, Ts)

    cur = np.zeros(K, dtype=np.float64)

    for i in range(K):
        prev_t = Ts[-1][i - 1] if i > 0 else left_border(t)
        next_t = Ts[-1][i + 1] if i < K - 1 else right_border(t)
        cur_t = Ts[-2][i]

        cur[i] = cur_t - s * (next_t - prev_t)

    return cur


def implicit_opposite(t, Ts):
    a = np.zeros(K - 1, dtype=np.float64)
    b = np.zeros(K, dtype=np.float64)
    c = np.zeros(K - 1, dtype=np.float64)
    d = np.zeros(K, dtype=np.float64)

    for i in range(K):
        b[i] = 1 + s + 2 * r
        d[i] = Ts[-1][i]
    d[0] += left_border(t) * (r + s)
    d[K - 1] += right_border(t) * r
    for i in range(K - 1):
        a[i] = - r - s
        c[i] = -r
    return solve_linear(a, b, c, d)


def implicit_nonopposite(t, Ts):
    a = np.zeros(K - 1, dtype=np.float64)
    b = np.zeros(K, dtype=np.float64)
    c = np.zeros(K - 1, dtype=np.float64)
    d = np.zeros(K, dtype=np.float64)

    for i in range(K):
        b[i] = 1 - s + 2 * r
        d[i] = Ts[-1][i]
    d[0] += left_border(t) * r
    d[K - 1] -= right_border(t) * (s - r)

    for i in range(K - 1):
        a[i] = -r
        c[i] = s - r

    return solve_linear(a, b, c, d)


def plot_img(cur_index, Ts, xs, y_min, y_max):
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.plot(xs, Ts[cur_index])

    ax.set_ylim(y_min, y_max)
    ax.set_xlim(0, 1e-4)
    

    fig.canvas.draw()
    image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
    image = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))

    plt.close(fig)

    return image



In [37]:
def run(method, filename, kappa_p, u_p, y_min='auto', y_max='auto'):
    global kappa, u
    global kappa = kappa_p
    u = u_p
    
    Ts = solve(method)

    print(len(Ts))

    xs = [
        a + i * dx
        for i in range(K)
    ]
    
    if y_min == 'auto':
        y_min = max(-1e13, np.min(np.array(Ts)))
    
    if y_max == 'auto':
        y_max = min(1e13, np.max(np.array(Ts)))


    frames = [
        plot_img(i, Ts, xs, y_min, y_max)
        for i in tqdm_notebook(range(len(Ts)))
        if i % 3 == 0
    ]
    imageio.mimsave(filename, frames, fps=24)

In [38]:
run(explicit_opposite, './kappa0/exp_opp.gif', 0, 5e-7)

502


In [39]:
run(explicit_opposite, './u0/exp_opp.gif', 8e-12, 0)

502


In [16]:
s

0.0125

In [13]:
r

0.0